## Model 1: Entry and exit rules for backtesting

**Model 1:** EMA50 crossover entry and EMA20 downtrend signal exit.

In [1]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
from datetime import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import sys
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip
from modelPEG import main


In [ ]:
# Call the main function and store the returned DataFrame
df = main()

In [ ]:
# Show dataframe
print(df.head())

In [10]:
# Example DataFrame setup
df_test = pd.DataFrame({
    'timestamp': pd.to_datetime([1,2,3,4,5,6,7,8,9,10], unit='D', origin='2022-01-01'),  # sample dates
    'symbol': ['AA','AA','AA','AA','AA','BB','BB','BB','BB','BB'],  
    'close': [10,9,11,11,9,100,110,120,125,150],
    'ema50': [9,10,10.5,10,10,95,115,110,120,130],
    'pegAlert': [1,0,0,0,0,1,0,0,0,0]
})

In [74]:
# Example DataFrame setup
df_test = pd.DataFrame({
    'timestamp': pd.to_datetime([1,2,3,4,5], unit='D', origin='2022-01-01'),  # sample dates
    'symbol': ['BB','BB','BB','BB','BB'],  
    'close': [100,110,120,130,150],
    'ema50': [95,115,110,120,130],
    'pegAlert': [1,0,0,0,0]
})

In [11]:
def calculate_cross_below_ema50(df):    
    # Sample DataFrame for illustration
    # Assuming 'symbol', 'date', 'close', 'open', 'ema20', 'ema50', and 'alert' columns exist in your DataFrame
    df['crossBelowEma50'] = 0  # Initialize the Entry column with 0

    # Reset index to ensure consecutive indexing within each symbol group
    df = df.reset_index(drop=True)

    # Iterate over rows where alert is 1
    for i, row in df[df['pegAlert'] == 1].iterrows():
        # Define the 5-row window after the alert
        end_index = min(i + 5, len(df))  # Ensure we don't go out of range
        for j in range(i + 1, end_index):
            # Check if 'close' is below 'ema50' within the 5-row window
            if df.at[j, 'close'] < df.at[j, 'ema50']:
                df.at[j, 'crossBelowEma50'] = 1  # Set 'Entry' to 1 in the row where condition is met
                break  # Stop searching within the 10-row window once condition is satisfied
    return df

# Display resulting DataFrame
# df_test_grouped = df_test.groupby('symbol', group_keys = False).apply(calculate_cross_below_ema50).reset_index(drop=True)
# print(df_test_grouped)

In [145]:
def calculate_entry(df):    
    # Initialize the Entry column with 0
    df['entryPos'] = 0

    # Reset index to ensure consecutive indexing within each symbol group
    df = df.reset_index(drop=True)  

    # Iterate over rows where 'crossBelowEma50' is 1
    for i, row in df[df['crossBelowEma50'] == 1].iterrows():
        # Define the 5-row window after the alert
        end_index = min(i + 5, len(df))  # Ensure we don't go out of range
        for j in range(i + 1, end_index):
            # Check if 'close' is above 'ema50' within the 5-row window
            if df.at[j, 'close'] > df.at[j, 'ema50']:
                delayed_index = j + 1  # Set entryPos 2 days after the condition is met
                if delayed_index < len(df):  # Check to avoid out of range
                    df.at[delayed_index, 'entryPos'] = 1
                break  # Stop searching within the 10-row window once condition is satisfied
    return df

# Apply the function by symbol
# df_test_grouped_2 = df_test_grouped.groupby('symbol').apply(calculate_entry).reset_index(drop=True)
# print(df_test_grouped_2)


In [12]:
def calculate_entry_2(df):    
    # Initialize the entryPos column with 0
    df['entryPos'] = 0

    # Reset index to ensure consecutive indexing within each symbol group
    df = df.reset_index(drop=True)  

    # Iterate over rows where 'crossBelowEma50' is 1
    for i, row in df[df['crossBelowEma50'] == 1].iterrows():
        # Define the 5-row window after the alert
        end_index = min(i + 5, len(df))  # Ensure we don't go out of range
        for j in range(i + 1, end_index):
            # Check if 'close' is above 'ema50' within the 5-row window
            if df.at[j, 'close'] > df.at[j, 'ema50']:
                # The immediate next row (j + 1) remains 0
                delayed_index_2 = j + 1  # Check 2 rows after the condition is met
                
                # Check for the condition two rows after j
                if delayed_index_2 < len(df) and df.at[delayed_index_2, 'close'] > df.at[delayed_index_2, 'ema50']:
                    df.at[delayed_index_2, 'entryPos'] = 1  # Set entryPos to 1 for j + 2
                
                break  # Stop searching within the 5-row window once the first condition is satisfied
    return df


In [13]:
def calculate_cross_below_ema50_for_symbols(df, symbols):
    # Initialize an empty list to hold DataFrames for each symbol
    df_list = []

    for symbol in symbols:
        # Filter the DataFrame for the current symbol
        df_symbol = df[df['symbol'] == symbol].copy()
        
        # Call the calculation function for the filtered symbol DataFrame
        df_symbol = calculate_cross_below_ema50(df_symbol)
        
        # Append the processed DataFrame to the list
        df_list.append(df_symbol)

    # Concatenate all symbol DataFrames into one final DataFrame
    df_final = pd.concat(df_list, ignore_index=True)
    
    return df_final

In [14]:
# List of symbols from dataframe
symbols_list = df_test['symbol'].unique()
print(symbols_list)

['AA' 'BB']


In [15]:

df = calculate_cross_below_ema50_for_symbols(df_test, symbols_list)
print(df)

   timestamp symbol  close  ema50  pegAlert  crossBelowEma50
0 2022-01-02     AA     10    9.0         1                0
1 2022-01-03     AA      9   10.0         0                1
2 2022-01-04     AA     11   10.5         0                0
3 2022-01-05     AA     11   10.0         0                0
4 2022-01-06     AA      9   10.0         0                0
5 2022-01-07     BB    100   95.0         1                0
6 2022-01-08     BB    110  115.0         0                1
7 2022-01-09     BB    120  110.0         0                0
8 2022-01-10     BB    125  120.0         0                0
9 2022-01-11     BB    150  130.0         0                0


In [16]:
def calculate_entry_for_symbols(df, symbols):
    # Initialize an empty list to hold DataFrames for each symbol
    df_list = []

    for symbol in symbols:
        # Filter the DataFrame for the current symbol
        df_symbol = df[df['symbol'] == symbol].copy()
        
        # Call the calculation function for the filtered symbol DataFrame
        df_symbol = calculate_entry_2(df_symbol)
        
        # Append the processed DataFrame to the list
        df_list.append(df_symbol)

    # Concatenate all symbol DataFrames into one final DataFrame
    df_final = pd.concat(df_list, ignore_index=True)
    
    return df_final

In [17]:
df = calculate_entry_for_symbols(df, symbols_list)
print(df)

   timestamp symbol  close  ema50  pegAlert  crossBelowEma50  entryPos
0 2022-01-02     AA     10    9.0         1                0         0
1 2022-01-03     AA      9   10.0         0                1         0
2 2022-01-04     AA     11   10.5         0                0         0
3 2022-01-05     AA     11   10.0         0                0         1
4 2022-01-06     AA      9   10.0         0                0         0
5 2022-01-07     BB    100   95.0         1                0         0
6 2022-01-08     BB    110  115.0         0                1         0
7 2022-01-09     BB    120  110.0         0                0         0
8 2022-01-10     BB    125  120.0         0                0         1
9 2022-01-11     BB    150  130.0         0                0         0
